In [87]:
import requests
from bs4 import BeautifulSoup
import time
import json
import csv

## Get a list of states Menards accepts

In [ ]:
url = "https://www.menards.com/store-details/locator.html"

payload = {}
headers = {}

response = requests.request("GET", url, headers=headers, data=payload)

In [22]:
soup = BeautifulSoup(response.text, 'html.parser')
states = soup.find(id="state-input").find_all('option')

states_list = [x['value'] for x in states]
states_list = list(filter(lambda x: len(x) > 0, states_list))
print(states_list)

['AL', 'AB', 'AK', 'AS', 'AZ', 'AR', 'BC', 'CA', 'CO', 'CT', 'DE', 'DC', 'FM', 'FL', 'GA', 'GU', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MB', 'MH', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NB', 'NH', 'NJ', 'NM', 'NY', 'NL', 'NC', 'ND', 'MP', 'NT', 'NS', 'NU', 'OH', 'OK', 'ON', 'OR', 'PW', 'PA', 'PE', 'PR', 'QC', 'RI', 'SK', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VI', 'VA', 'WA', 'WV', 'WI', 'WY', 'YT']


## Create a dictionary of States to Cities Menards accept

In [44]:
clean_url = "https://www.menards.com/store-details/menard-cities-by-state.ajx?state="

state_cities = {}

for state_abbr in states_list:
    time.sleep(2)
    appeneded_url = clean_url+state_abbr
    state_cities[state_abbr] = requests.request("GET", appeneded_url).text

In [73]:
cleaned_state_cities = {x:json.loads(y) for (x,y) in state_cities.items()}
cleaned_state_cities = {x:y for (x,y) in cleaned_state_cities.items() if y['success']}
for (x,y) in cleaned_state_cities.items():
    del cleaned_state_cities[x]['success']
cleaned_state_cities = {x:y['cities'] for (x,y) in cleaned_state_cities.items()}


print(len(cleaned_state_cities))

15


## Get list of Menards store by state and city

In [81]:

clean_url = "https://www.menards.com/store-details/locate-stores-by-address.ajx?address="
stores = {}

for (state,cities) in cleaned_state_cities.items():
    for city in cities:
        time.sleep(2)
        appeneded_url = clean_url+city+',+'+state
        stores[(state,city)] = requests.request("GET", appeneded_url)

# response = requests.request("GET", url)

# print(response.text)


In [135]:
cleaned_stores={ x:y.text for (x,y) in stores.items()}
cleaned_stores={x:json.loads(y)["storeResults"] for (x,y) in cleaned_stores.items()}

In [153]:
# cleaned_stores[('IL','ANTIOCH')]
stores = {}
for (x,y) in cleaned_stores.items():
    for store in y:
        stores[store['number']] = store

print((stores))

{3211: {'number': 3211, 'name': 'ANTIOCH', 'street': '369 E IL ROUTE 173', 'city': 'ANTIOCH', 'state': 'IL', 'zip': '60002', 'latitude': '42.468373', 'longitude': '-88.068651', 'distance': 37.3, 'open': True, 'services': [{'displayName': 'Appliances', 'image': 'assets/images5/storeLocator/appAvailLogo.png', 'categoryId': 5570}, {'displayName': 'Garden Center', 'image': 'assets/images5/storeLocator/gardenLogo.png', 'categoryId': 12577}, {'displayName': 'Grocery', 'image': 'assets/images5/storeLocator/groceryPet.png', 'categoryId': 19741}, {'displayName': 'Propane Fueling', 'image': 'assets/images5/storeLocator/propaneFueling.png', 'categoryId': 13167}, {'displayName': 'Rental Center', 'image': 'assets/images5/storeLocator/rentalCenter.png', 'categoryId': 13371}, {'displayName': 'Truck Rental', 'image': 'assets/images5/storeLocator/truckRental.png', 'categoryId': 13371}]}, 3119: {'number': 3119, 'name': 'FOX LAKE', 'street': '1400 S US HIGHWAY 12', 'city': 'FOX LAKE', 'state': 'IL', 'zip

save to file so I don't have to run the requests again

In [154]:
with open("Menards_stores.json", "w") as outfile:
    json.dump(stores, outfile)

In [ ]:
menards_stores = json.load(open('Menards_stores.json'))

In [166]:
freq_dict = {}
for (store_num,info) in menards_stores.items():
    freq_dict[info['state']] = freq_dict.get(info['state'],0)+1
print(freq_dict)

{'IL': 60, 'WI': 44, 'IN': 37, 'IA': 21, 'MO': 20, 'KY': 5, 'MI': 32, 'OH': 32, 'NE': 14, 'WV': 3, 'KS': 9, 'MN': 40, 'ND': 7, 'SD': 8, 'WY': 3}


In [ ]:
with open("stores_per_state.json", "w") as outfile:
    json.dump(freq_dict, outfile)

Data is now used in Visualization.ipynb 